#### 모듈 설치

In [ ]:
%pip install python-dotenv requests boto3 psycopg2-binary

#### 환경변수 로드 및 저장할 폴더 준비

In [2]:
import os
from dotenv import load_dotenv
from pathlib import Path

# 환경 변수 로드
load_dotenv()

API_KEY = os.getenv("KOBIS_API_KEY")

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("AWS_REGION")
S3_BUCKET_NAME = os.getenv("S3_BUCKET_NAME")

REDSHIFT_HOST = os.getenv("REDSHIFT_HOST")
REDSHIFT_PORT = os.getenv("REDSHIFT_PORT")
REDSHIFT_DB = os.getenv("REDSHIFT_DB")
REDSHIFT_USER = os.getenv("REDSHIFT_USER")
REDSHIFT_PASSWORD = os.getenv("REDSHIFT_PASSWORD")
IAM_ROLE_ARN = os.getenv("IAM_ROLE_ARN")

# 저장할 폴더 준비
project_folder = project_folder = Path.cwd().parent
raw_data_dir = project_folder / "raw_data" / "movieInfo"    # 영화 상세 정보
raw_data_dir.mkdir(exist_ok=True)

#### 데이터 수집

In [7]:
import glob
import json
from pathlib import Path

project_folder = Path.cwd().parent
folder_path = project_folder / "raw_data" / "boxoffice"    # 일별 박스오피스
file_pattern = str(folder_path / "dailyBoxOffice_*.json")
json_files = glob.glob(file_pattern)

if not json_files:
    print(f"경고: {file_pattern} 패턴의 파일을 찾을 수 없습니다.")

distinct_movie_cds = set()

for file_path in json_files:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip(): # 빈 줄이 아닌 경우에만 처리
                    data = json.loads(line)
                    distinct_movie_cds.add(data.get("movieCd"))
    except Exception as e:
        print(f"파일 {file_path} 처리 중 오류 발생: {e}")

print(distinct_movie_cds)

{20256526, 20040590, 20242964, 20241941, 20256149, 20243864, 20256281, 19950366, 20256034, 20254501, 20257586, 20256185, 20257083, 20256701, 20256702, 20245055, 20246333, 20256962, 20254275, 20257095, 20256071, 20256202, 20247245, 20240206, 20233039, 20257229, 20161872, 20030047, 20256864, 20256741, 20020328, 20253289, 20243561, 20245485, 20231024, 20249072, 20210546, 20256757, 20228988, 20244606}


In [ ]:
import requests
import json
import time # API 호출 간격을 위한 time 모듈 추가
from datetime import datetime
from pathlib import Path # Path 객체 사용을 가정하여 추가


# 데이터 수집
base_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json" # 영화 상세 정보

for i, movieCd in enumerate(distinct_movie_cds):
    params = {"key": API_KEY, "movieCd": movieCd}

    try:
        res = requests.get(base_url, params=params, timeout=10)
        res.raise_for_status()
        data = res.json()

        movie_info = data['movieInfoResult']['movieInfo']

        genres_list = movie_info.get('genres', [])
        genre_names = [g['genreNm'] for g in genres_list if g.get('genreNm')]
        genre = genre_names[0] if genre_names else ""

        open_dt_raw = movie_info.get('openDt', '')
        open_dt_formatted = None
        if open_dt_raw and len(open_dt_raw) == 8:
            try:
                # '2025901' -> '2025-09-01'로 변환
                open_dt_formatted = datetime.strptime(open_dt_raw, '%Y%m%d').strftime('%Y-%m-%d')
            except ValueError:
                pass # 변환 실패 시 None 유지

        # Redshift 테이블 스키마에 맞게 데이터 타입 변환 및 가공
        movie_data = {
            "movieCd": int(movie_info.get("movieCd")),  # INT
            "movieNm": movie_info.get("movieNm"),       # VARCHAR
            "openDt": open_dt_formatted,                # DATE (YYYY-MM-DD 형식 문자열)
            "genre": genre                              # VARCHAR (장르 여러개일시 첫번째 값)
        }

        file_path = raw_data_dir / f"movieInfo_{movieCd}.json"
        with open(file_path, "w", encoding="utf-8") as f:
            json_line = json.dumps(movie_data, ensure_ascii=False)
            f.write(json_line + '\n') 

        print(f"[{i+1}] movieInfo_{movieCd} 저장 완료 (JSON Lines 형식)")

    except Exception as e:
        print(f"movieInfo_{movieCd} 데이터 수집 실패: {e}")

[1] movieInfo_20256526 저장 완료 (JSON Lines 형식)
[2] movieInfo_20040590 저장 완료 (JSON Lines 형식)
[3] movieInfo_20242964 저장 완료 (JSON Lines 형식)
[4] movieInfo_20241941 저장 완료 (JSON Lines 형식)
[5] movieInfo_20256149 저장 완료 (JSON Lines 형식)
[6] movieInfo_20243864 저장 완료 (JSON Lines 형식)
[7] movieInfo_20256281 저장 완료 (JSON Lines 형식)
[8] movieInfo_19950366 저장 완료 (JSON Lines 형식)
[9] movieInfo_20256034 저장 완료 (JSON Lines 형식)
[10] movieInfo_20254501 저장 완료 (JSON Lines 형식)
[11] movieInfo_20257586 저장 완료 (JSON Lines 형식)
[12] movieInfo_20256185 저장 완료 (JSON Lines 형식)
[13] movieInfo_20257083 저장 완료 (JSON Lines 형식)
[14] movieInfo_20256701 저장 완료 (JSON Lines 형식)
[15] movieInfo_20256702 저장 완료 (JSON Lines 형식)
[16] movieInfo_20245055 저장 완료 (JSON Lines 형식)
[17] movieInfo_20246333 저장 완료 (JSON Lines 형식)
[18] movieInfo_20256962 저장 완료 (JSON Lines 형식)
[19] movieInfo_20254275 저장 완료 (JSON Lines 형식)
[20] movieInfo_20257095 저장 완료 (JSON Lines 형식)
[21] movieInfo_20256071 저장 완료 (JSON Lines 형식)
[22] movieInfo_20256202 저장 완료 (JSON Lines 형

#### S3 에 업로드

In [5]:
print(raw_data_dir)

/Users/kang/Team4_4Ward/BI-Dashboard/raw_data/movieInfo


In [6]:
import boto3

# S3 업로드
if any(raw_data_dir.glob("*.json")):
    s3 = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name=AWS_REGION
    )

    for file in raw_data_dir.glob("movieInfo_*.json"):
        file_name = f"raw_data/movieInfo/{file.name}"   # 영화 상세 정보
        try:
            s3.upload_file(str(file), S3_BUCKET_NAME, file_name)
            print(f"[success] S3 업로드 완료: {file_name}")
        except Exception as e:
            print(f"[fail] S3 업로드 실패 ({file_name}): {e}")
else:
    print("[error] 업로드할 파일이 없습니다.")

[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20210546.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20256202.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20256701.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20256185.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20240206.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20256281.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20256757.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20256741.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20256864.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20256034.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20257229.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20254501.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20256526.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_20253289.json
[success] S3 업로드 완료: raw_data/movieInfo/movieInfo_19950366.json
[success] S3 업로드 완료: raw_data/movieInfo/